In [7]:
print()
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


In [8]:
import gym
from gym.wrappers import Monitor
import glob
import io
import base64

from gym import wrappers
from IPython import display
import matplotlib
import matplotlib.pyplot as plt
import gym
import numpy as np
from collections import deque
import tensorflow as tf
from tensorflow.keras import layers,models

def query_environment(name):
  env = gym.make(name)
  spec = gym.spec(name)
  print(f"Action Space: {env.action_space}")
  print(f"Observation Space: {env.observation_space}")
  print(f"Max Episode Steps: {spec.max_episode_steps}")
  print(f"Nondeterministic: {spec.nondeterministic}")
  print(f"Reward Range: {env.reward_range}")
  print(f"Reward Threshold: {spec.reward_threshold}")

In [31]:
MEMORYLEN=int(1e5)
# MEMORYLEN=int(100)
BATCHSIZE=64
EPOCHS=1
UPDATE_EVERY = 4


class DQNAgent():
    def __init__(self,actions=4,obs=8):
        self.actions=actions
        self.observations=obs
        self.model=self.load_model()
        self.target_model = self.load_model()
        self.memory=deque(maxlen=MEMORYLEN)
        self.gamma=0.99
        self.i=0
        self.tau = .125
           
    def play(self,observation,epsilon):
        if (len(self.memory)<MEMORYLEN):
            action=np.random.randint(low=0,high=self.actions)
            return action
        else:
            if np.random.random()>epsilon:
                action=self.model_predictions(observation)
            else:
                action=np.random.randint(low=0,high=self.actions)
            
            return action
                
         
            
    def step(self,state, action, reward, next_state, done):
        self.memory.append([state, action, reward, next_state, done])
        self.i+=1
        if (len(self.memory)>=BATCHSIZE):
            if (self.i % UPDATE_EVERY==0):
                self.train_model()
                self.target_train()
        pass
 
    
    def train_model(self):
#         data=np.random.choice(self.memory,BATCHSIZE)
        rnd_indices = np.random.choice(len(self.memory), size=BATCHSIZE)
        data=np.array(self.memory)[rnd_indices]
        
        for sample in data:
            state, action, reward, new_state, done = sample
            target = self.target_model.predict(state.reshape(1,-1))
            if done:
                target[0][action] = reward
            else:
                Q_future = max(self.target_model.predict(new_state.reshape(1,-1))[0])
                target[0][action] = reward + Q_future * self.gamma
            self.model.fit(state.reshape(1,-1), target, epochs=1, verbose=0)
            



#         np.random.shuffle(data)
        # state, action, reward, next_state, done=np.stack(data[:,0]),np.stack(data[:,1]),np.stack(data[:,2]),np.stack(data[:,3]),np.stack(data[:,4])
        # q_expected_for_next_state=np.max(self.model.predict(next_state),axis=1)
        # q_target_update=reward + (self.gamma* q_expected_for_next_state*(1-done))
        
        # target_qtable = self.model.predict(state)
        # for i,target_f in enumerate(target_qtable):
        #     action_toupdate=action[i]
        #     target_f[action_toupdate] = q_target_update[i]
        #     target_qtable[i]=target_f
        
        # self.model.fit(state,target_qtable,epochs=EPOCHS,batch_size=BATCHSIZE,verbose=0)
        
        pass
    def model_predictions(self,observation):
        pred=self.model.predict(observation.reshape(1,-1))
        pred=np.argmax(pred)
        return pred
        
    def load_model(self):
        num_input = layers.Input(shape=(self.observations, ))
        x = layers.Dense(64)(num_input)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.1)(x)
        x = layers.Dense(64, activation="relu")(x)
        x = layers.Dropout(0.1)(x)
        x = layers.BatchNormalization()(x)
        y = layers.Dense(self.actions, activation="linear")(x)
        model = models.Model(inputs=num_input, outputs=y)
        model.compile(loss="mean_squared_error",optimizer=tf.keras.optimizers.Adam(lr=5e-4))
        model.summary()
        
        return model
    def target_train(self):
        weights = self.model.get_weights()
        target_weights = self.target_model.get_weights()
        for i in range(len(target_weights)):
            target_weights[i] = weights[i] * self.tau + target_weights[i] * (1 - self.tau)
        self.target_model.set_weights(target_weights)
    

In [19]:
import time
from tqdm import tqdm
starttime=time.time()
scores = []                        # list containing scores from each episode
scores_window = deque(maxlen=100)  # last 100 scores
n_episodes=2000
agent=DQNAgent()

max_t=1000
eps_start=1.0
eps_end=0.01
eps_decay=0.995



eps = eps_start
env=gym.make('LunarLander-v2')
eps_history=[]
for i_episode in tqdm(range(1, n_episodes+1),total=n_episodes+1):
    state = env.reset()
    score = 0
    while True:
        action = agent.play(state,eps)
        next_state, reward, done, _ = env.step(action)
        agent.step(state, action, reward, next_state, done)
        state = next_state
        score += reward
        if done:
            break 
    scores_window.append(score)       # save most recent score
    scores.append(score)              # save most recent score
    eps = max(eps_end, eps_decay*eps)
    eps_history.append(eps)
    if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
    if np.mean(scores_window)>=200.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            
            break
endtime= time.time() 
print(endtime-starttime)
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

0%|          | 0/2001 [00:00<?, ?it/s]Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 8)]               0         
_________________________________________________________________
dense_12 (Dense)             (None, 64)                576       
_________________________________________________________________
batch_normalization_8 (Batch (None, 64)                256       
_________________________________________________________________
dropout_8 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_9 (Dropout)          (None, 64)                0         
_________________________________________________________________
batch_normalization_9

ValueError: Error when checking input: expected input_6 to have shape (8,) but got array with shape (1,)

In [ ]:
scores[-10:]

In [25]:
rnd_indices = np.random.choice(len(agent.memory), size=BATCHSIZE)
data=np.array(agent.memory)[rnd_indices]

for sample in data:
    state, action, reward, new_state, done = sample
    break
state

array([ 0.12217007,  0.50383115,  0.22293237, -1.1520833 , -0.05542531,
       -0.00562378,  0.        ,  0.        ], dtype=float32)

In [30]:
agent.model.predict(state.reshape(1,-1))[0]

array([ 0.1600027 , -0.30031613,  0.20209691, -0.12400401], dtype=float32)